In [9]:
# Import libraries
from io import StringIO
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os

# load data directories
lunar_test_data_directory = './space_apps_2024_seismic_detection/data/lunar/test/data'
lunar_training_data_directory = './space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA/'

mars_test_data_directory = './space_apps_2024_seismic_detection/data/mars/test/data'
mars_training_data_directory = './space_apps_2024_seismic_detection/data/mars/training/data'

lunar_training_file = lunar_training_data_directory + 'xa.s12.00.mhz.1970-01-19HR00_evid00002.csv'

l_training_df = pd.read_csv(lunar_training_file)
# l_train_df = pd.read_csv()


,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),velocity(m/s)
